In [1]:
import os

import openmm as mm
from openmm.app import *
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
pdb = PDBFile('brs_full.pdb') #Sistema Vacio

In [3]:
forcefield = ForceField('amber14-all.xml', 'implicit/gbn2.xml') #Se indica el solvente implicito

In [25]:
# Listado de modelos solvente implicito 

#implicit/hct.xml
#implicit/obc1.xml
#implicit/obc2.xml
#implicit/gbn.xml
#implicit/gbn2.xml

In [4]:
# El modelo de solvente implicito de AMBER no es compatible con nonbondedMethod

system = forcefield.createSystem(pdb.topology, soluteDielectric=1.0, solventDielectric=80.0, nonbondedCutoff=1.2*unit.nanometer, switchDistance=0.9*unit.nanometer, constraints=HBonds)

In [5]:
system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fe9f4f91060> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fe9f4f91150> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fe9f4f911e0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fe9f4f91270> >,
 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fe9f4f91300> >,
 <openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7fe9f4f91390> >]

Del listado se observa que la ultima fuerza corresponde al modelo de solvente implicito

In [6]:
implicit = system.getForces()[5]

In [ ]:
implicit.getNumPerParticleParameters()

7

In [ ]:
implicit.getPerParticleParameterName(2)

'sr'

In [24]:
# Impresion de los nombres de los parametros que tiene el modelo de solvente implicito seleccionado

for ii in range(implicit.getNumPerParticleParameters()):
    print(implicit.getPerParticleParameterName(ii))

charge
or
sr
alpha
beta
gamma
radindex


In [23]:
# Creacion de listas con los parametros involucrados en el modelo de solvente implicito seleccionado

charge  = []
or_     = []
sr      = []
alpha   = []
beta    = []
gamma   = []
radinex = []

for ii in range(implicit.getNumParticles()):
    aa, bb, cc, dd, ee, ff, gg = implicit.getParticleParameters(ii)
    charge.append(aa)
    or_.append(bb)
    sr.append(cc)
    alpha.append(dd)
    beta.append(ee)
    gamma.append(ff)
    radinex.append(gg)

In [6]:
step_size   = 0.002*unit.picoseconds
temperature = 300*unit.kelvin
friction    = 1.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

In [7]:
platform_name = 'CUDA'
platform      = mm.Platform.getPlatformByName(platform_name)

In [8]:
simulation = Simulation(pdb.topology, system, integrator, platform)

In [9]:
simulation.context.setPositions(pdb.positions)

In [10]:
simulation.minimizeEnergy()

In [11]:
# Parámetros de la simulación.

simulation_time = 0.1*unit.nanosecond
saving_time     = 10.0*unit.picoseconds

In [12]:
n_steps_per_period = int(saving_time/step_size) # número de pasos entre frame de guardado
n_periods          = int(simulation_time/saving_time) # número de frames guardados

In [13]:
n_particles  = pdb.topology.getNumAtoms()

In [14]:
# Creación de arrays reporteros del tiempo, la posición y la velocidad.

times                  = np.zeros([n_periods], np.float32) * unit.picoseconds
positions              = np.zeros([n_periods, n_particles, 3], np.float32) * unit.angstroms
potential_energies     = np.zeros([n_periods], np.float32) * unit.kilocalories_per_mole

In [15]:
# Almacenamiento en reporteros de las condiciones iniciales para tiempo 0

state                   = simulation.context.getState(getPositions=True, getEnergy=True)

times[0]                = state.getTime()
positions[0]            = state.getPositions()
potential_energies[0]   = state.getPotentialEnergy()

In [16]:
# Ejecuto el bucle sobre el número de periodos que vamos a simular

for ii in tqdm(range(1, n_periods)):
    simulation.context.getIntegrator().step(n_steps_per_period)
    state                    = simulation.context.getState(getPositions=True, getEnergy=True)
    times[ii]                = state.getTime()
    positions[ii]            = state.getPositions()
    potential_energies[ii]   = state.getPotentialEnergy()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:10<00:00, 14.47s/it]
